<a href="https://colab.research.google.com/github/ProjectFace2/reservoirOutflow/blob/master/Ensemble_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
from numpy import concatenate
from datetime import datetime
import matplotlib as mpl
from matplotlib import pyplot
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
import fbprophet as fb
import pickle

dataset 

In [0]:
df = pd.read_csv('datasets/allYearHarangi.csv',index_col=0)
df.drop(["Present Storage(TMC)",'Reservoir Level(TMC)','Outflow'],axis = 1,inplace = True)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(df.values)
reshaped=pd.DataFrame({'Inflow':scaled[:,0],'MADIKERI':scaled[:,1],'SOMWARPET':scaled[:,2],'VIRAJPET':scaled[:,3]})


In [0]:
'''
In this method, we use all the lag days and lead days and construct a dataframe, where
the lag and lead days are represented in a single row
Using these features we can predict the class of the present day with the data from the previous days
This holds good for all the rows in the dataframe
'''

#dataset preparation with lag
def series_to_supervised(data, lag_days=1, lead_days=1, dropnan=True):
    no_of_features = 1 if type(data) is list else data.shape[1]
    print(no_of_features)
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(lag_days, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(no_of_features)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, lead_days):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(no_of_features)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(no_of_features)]
# put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [122]:
x=reshaped.iloc[-5:-1,:].values
x=np.append(x,[[0.0,0.0,0.0,0.0]],axis=0)
test=series_to_supervised(x,4,0)
test

4


,var1(t-4),var2(t-4),var3(t-4),var4(t-4),var1(t-3),var2(t-3),var3(t-3),var4(t-3),var1(t-2),var2(t-2),var3(t-2),var4(t-2),var1(t-1),var2(t-1),var3(t-1),var4(t-1)
4,0.001322,0.0,0.0,0.0,0.001258,0.0,0.0,0.0,0.001309,0.0,0.0,0.0,0.001271,0.0,0.0,0.0


In [0]:
def predict(queryset,date):
  '''based on query set ,and given date range predicts result for given date with multiple models and returns ensembled result 
    query set can be single row or multiple row set, in case of multiple set use dataframe range with set of prediction seeking dates 
  '''
  # lstm prediction
  n_hours = 4
  n_features = 4
  test_x=queryset.values
  lstm_test_X = test_x.reshape((test_x.shape[0], n_hours, n_features))
  with open('models/lstmInf_forecast_model_lag4.pckl', 'rb') as fin:
    lstm_lag4_model = pickle.load(fin)
  lstm_inv_yhat=lstm_lag4_model.predict(lstm_test_X)
  test_X = lstm_test_X.reshape((lstm_test_X.shape[0], n_hours*n_features))
  inv_yhatx = concatenate((yhat, test_X[:, -3:]), axis=1)
  inv_yhaty = scaler.inverse_transform(inv_yhatx)
  lstm_res = inv_yhaty[:,0]

  # fb prophet prediction
  fb_test=queryset
  varList=fb_test.columns.tolist()
  varList.insert(0,'ds')
  fb_test['ds']=date
  fb_test_X = pd.DataFrame(fb_test[varList])
  with open('models/fbInf_forecast_model_lag4.pckl', 'rb') as fin:
    lstm_lag4_model = pickle.load(fin)
  proph_inv_yhat = proph_lag4_model.predict(fb_test_X).values
  proph_res = fb_inv_yhat[:,-1]

  # ensembled prediction
  result=((fb_inv_yhat[:,-1])+(inv_yhat))/2 #as of now it is average
  return result